In [ ]:
import pandas as pd
from datetime import datetime
import panel as pn

pn.extension('tabulator')  # for interactive table

# --- Load CSV from GitHub ---
github_url = "https://raw.githubusercontent.com/FrankCoRa/room-availability-dashboard/main/available_rooms.csv"

try:
    df = pd.read_csv(github_url)
except pd.errors.ParserError:
    # fallback: try tab-separated
    df = pd.read_csv(github_url, sep="\t")

# --- Clean column headers ---
df.columns = df.columns.str.strip()  # removes leading/trailing spaces
print("Columns detected:", df.columns.tolist())  # optional, useful for debugging

# --- Apply 'Hide in Scheduling' filter if exists ---
if "Hide in Scheduling" in df.columns:
    df["Hide in Scheduling"] = df["Hide in Scheduling"].astype(str).str.strip().str.lower()
    df = df[df["Hide in Scheduling"] == "false"]
else:
    print("Warning: 'Hide in Scheduling' column not found. Skipping this filter.")

# --- Helper: Convert string to datetime.time ---
def to_time(t):
    for fmt in ("%I:%M%p", "%H:%M"):
        try:
            return datetime.strptime(str(t).strip(), fmt).time()
        except ValueError:
            continue
    raise ValueError(f"Invalid time format: {t}")

# --- Check if requested time is within availability ---
def is_within(time_range, start, end):
    return start <= time_range[0] and end >= time_range[1]

# --- Main Filtering Function ---
def available_rooms(start_time, end_time, weekdays, building=None):
    start_time = to_time(start_time)
    end_time = to_time(end_time)
    
    if isinstance(weekdays, str):
        weekdays = [weekdays]
    weekdays = [day.lower() for day in weekdays]
    
    filtered_df = df[df["Weekday"].str.lower().isin(weekdays)]
    if building:
        filtered_df = filtered_df[filtered_df["Building"].str.lower() == building.lower()]
    
    available = []
    for room, group in filtered_df.groupby("Room"):
        valid_days = 0
        for _, row in group.iterrows():
            try:
                t_start, t_end = [to_time(x) for x in row["Availability"].split("-")]
            except Exception:
                continue
            if is_within((start_time, end_time), t_start, t_end):
                valid_days += 1
        if valid_days == len(weekdays):
            room_info = group.iloc[0][["Room", "Building", "Capacity", "Type", "Features"]].to_dict()
            available.append(room_info)
    
    return pd.DataFrame(available)

# --- PANEL WIDGETS ---
start_picker = pn.widgets.TimePicker(name='Start Time', value='09:00')
end_picker = pn.widgets.TimePicker(name='End Time', value='17:00')

weekday_opts = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_check = pn.widgets.CheckBoxGroup(name='Select Weekdays', value=['Monday'], options=weekday_opts, inline=True)

building_list = sorted(df["Building"].dropna().unique().tolist())
building_dropdown = pn.widgets.Select(name='Building', options=['All'] + building_list)

# --- PANEL CALLBACK ---
@pn.depends(start_picker, end_picker, weekday_check, building_dropdown)
def update_table(start_time, end_time, weekdays, building):
    building_filter = None if building == 'All' else building

    def format_time(t):
        if hasattr(t, 'strftime'):
            return t.strftime("%I:%M%p")
        t = str(t).strip()
        for fmt in ("%I:%M%p", "%H:%M"):
            try:
                return datetime.strptime(t, fmt).strftime("%I:%M%p")
            except ValueError:
                continue
        raise ValueError(f"Invalid time format: {t}")

    start_str = format_time(start_time)
    end_str = format_time(end_time)

    result_df = available_rooms(start_str, end_str, weekdays, building=building_filter)

    if result_df.empty:
        return pn.pane.Markdown("⚠️ No rooms available for the selected criteria.")
    else:
        return pn.widgets.Tabulator(
            result_df,
            pagination='remote',
            page_size=10,
            sizing_mode='stretch_width'
        )

# --- PANEL LAYOUT ---
dashboard = pn.Column(
    pn.pane.Markdown("## 🏫 Room Availability Finder"),
    pn.Row(start_picker, end_picker),
    weekday_check,
    building_dropdown,
    pn.layout.Divider(),
    update_table
)

# --- RUN APP IN BINDER / LOCAL ---
if __name__ == "__main__":
    pn.serve(dashboard, show=True, port=5000, websocket_origin="*")


Column
    [0] Markdown(str)
    [1] Row
        [0] TimePicker(name='Start Time', value='09:00')
        [1] TimePicker(name='End Time', value='17:00')
    [2] CheckBoxGroup(inline=True, name='Select Weekdays', options=['Monday', 'Tuesday', ...], value=['Monday'])
    [3] Select(name='Building', options=['All', 'BRONX', ...], value='All')
    [4] Divider()
    [5] ParamFunction(function, _pane=Tabulator, defer_load=False)